In [2]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import os
import shutil
import cv2
from tqdm import trange

#import glob2

from scipy.io import loadmat
import scipy
#import hdf5storage
import csv
from tqdm import trange

import glob

Autosaving every 180 seconds


In [3]:
#  
def get_animal_ids(frame,
                  chain_ids,
                  root_dir,
                  vals,
                  classes,
                  comments=False):
    #frame = 988
    #print ("FRAME: ", frame)
    #print ("classes: ", classes.shape)
    
    dirs = chain_ids[frame]

    if comments:
        print ("DIRS: ", dirs)
    
    animal_ids = []
    animal_ids2 = []
    threshold_purity1 = 0.5
    threshold_purity2 = 0.7

    # HUNGARIAN ALGORITHM BASED SEARCH FOR MOST CORRECT IDS
    best_ids_cts = np.zeros((5,5),'float32')
    animals3 = []
    for ctr_dir, dir_ in enumerate(dirs):
        #print ("2nd Loop: ", dir_)
        if np.isnan(dir_)==False:
            # find the classifier values for that segment
            idx = np.where(vals==dir_)[0]

            # find predicted id
            fnames = np.sort(glob.glob(root_dir+str(int(dir_))+'/*.npy',recursive = False))

            ctr_chain = 0
            for fname in fnames:
                if ('frame_'+str(frame).zfill(7)) in fname:
                     break 
                ctr_chain+=1
        
            animal_id3 = classes[idx][ctr_chain]
            animals3.append(animal_id3)
            
            idx5 = np.where(classes[idx]<1E10)[0]
            classes_local = classes[idx[idx5]]
            
            classes_unique = np.unique(classes_local, return_counts=True)
            #print ("best id: ", best_id)
            
            best_ids_cts[ctr_dir, classes_unique[0]]=classes_unique[1]/idx.shape[0]
        else:
            animals3.append(None)
    
    # FIRST LOOP TO CHECK ANY GREAT MATCHES AND ZERO OUT EVERYTHING ELSE
    for k1 in range(5):
        temp = best_ids_cts[k1]
        max_val = np.max(temp)
        if max_val>threshold_purity2:
            idx8 = np.argmax(temp)
            best_ids_cts[:,idx8]=0
            best_ids_cts[k1,idx8]=max_val
    if comments:
        print (best_ids_cts)
    # RENORMALIZE PROBABILITIES AFTER ZEROING OUT
    for k1 in range(5):
        sum_ = best_ids_cts[k1].sum()
        if sum_>0:
            best_ids_cts[k1]*= 1./sum_
    if comments:
        print (best_ids_cts)    
        
    # FINAL LOOP TO GRAB WHATEVER IS LEFT
    for k1 in range(5):
        max_val = np.max(best_ids_cts[k1])
        if max_val==0: # or max_val<threshold_purity1:
            animal_ids.append(None)
            animal_ids2.append(animals3[k1])
            continue
        else:
            idx9 = np.argmax(best_ids_cts[k1])
            #print ("idx9: ", idx9, classes_unique[0])
            animal_ids.append(idx9)
            animal_ids2.append(animals3[k1])
            
            best_ids_cts[:,idx9]= 0
            # must zero out all the others
            
    if comments:
        print (best_ids_cts)           
    return animal_ids, animal_ids2, dirs

def get_animal_ids_function(fname_classification,
                   fname_traces_inferences,
                   chain_ids,
                   root_dir,
                   comments,
                   n_networks):
    
    fname_out = os.path.split(fname_classification)[0]+"/animal_arrays.npy"
    if os.path.exists(fname_out)==False:

        data = np.load(fname_classification, allow_pickle=True)
        vals = data['vals']
        frames = np.hstack(data['frames_array'])
        print ("# of lableed frames: ", vals.shape)
        classes = data['classes']

        reassembled = np.load(fname_traces_inferences)

        #reassembled = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/pickle/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz')
        tracesx_re = reassembled['tracesx'].T
        tracesy_re = reassembled['tracesy'].T
        print ("Lenght of tracessx: ", tracesx_re.shape)

        start = 1
        end = tracesx_re.shape[0]

        # INITIALIZE ARRAYS TO BE SAVED
        animal_ids_hungarian = np.zeros((tracesx_re.shape[0], n_networks), 'float32')
        animal_arrays = np.zeros((n_networks, tracesx_re.shape[0], 2), 'float32')
        for n in trange(start,end, 1):
            #ret, frame = original_vid.read()

            # animal_ids is the hungarian output;
            # animal_ids is the CNN output
            animal_ids, animal_ids2, dirs = get_animal_ids(n,
                                                        chain_ids,
                                                        root_dir,
                                                        vals,
                                                        classes,
                                                        comments)

            #print ("animal_ids: ", animal_ids)
            for k in range(0, 14*n_networks,14):
                y = tracesx_re[n,k:k+14]
                x = tracesy_re[n,k:k+14]

                idx = np.where(x!=0)[0]
                x = x[idx]
                y = y[idx]
                if x.shape[0]==0:
                    x_ave = None
                    y_ave = None
                else:
                    x_ave = np.nanmean(x)
                    y_ave = np.nanmean(y)

                if animal_ids[k//14] is None:
                    continue
                animal_arrays[animal_ids[k//14], n] = x_ave, y_ave

            animal_ids_hungarian[n]=animal_ids 

        np.save(fname_out, animal_arrays)
        np.save(fname_out[:-4]+"_animal_ids_hungarian.npy", animal_ids_hungarian)
    else:
        animal_arrays = np.load(fname_out)
        animal_ids_hungarian = np.load(fname_out[:-4]+"_animal_ids_hungarian.npy")

    print ("DONE making animal_arrays: ", animal_arrays.shape)

    return animal_arrays, animal_ids_hungarian

def get_single_feature_traces(fname_classification,
                              fname_traces_inferences,
                              animal_ids_hungarian,
                              n_networks,
                              features):
    
    fname_out = os.path.split(fname_classification)[0] + "/locs_array.npy"
    if os.path.exists(fname_out)==False:
    
        reassembled = np.load(fname_traces_inferences)

        #reassembled = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/pickle/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz')
        tracesx_re = reassembled['tracesx'].T
        tracesy_re = reassembled['tracesy'].T

        locs_array = np.zeros((n_networks, animal_ids_hungarian.shape[0],2),'float32')+np.nan
        
        for p in ids:
            # Plot spine1 location for each animal
            #locs = np.zeros(animal_ids_hungarian.shape[0],2,'float32')
            for k in range(1, animal_ids_hungarian.shape[0]):
                idxb = np.where(animal_ids_hungarian[k]==p)[0]
                if idxb.shape[0]>0:
                    tempx = tracesx_re[k,features+idxb*14]
                    #print ("tempx: ", tempx.shape)
                    tempy = tracesy_re[k, features+idxb*14]
                    locs_array[p,k]= tempx, tempy

        np.save(fname_out, locs_array)
    
    else:
        locs_array = np.load(fname_out)
        
    return locs_array


def cleanup_traces(locs_array, 
                  animal_id,
                  velocity_max,
                  velocity_min,
                  velocity_rel_max,
                  min_seg_len):
    
    data = locs_array[animal_id].copy()
    
    # REMOVE GARBAGE / FAILED PREDICTION
    idx = np.where(data[:,0]==0.)[0]
    #print (" ZERO REMOVAL: ", idx.shape)
    data[idx,0]=np.nan
    data[idx,1]=np.nan

    # DEL SHORT SEGS 
    idx = np.where(np.isnan(data[:,0])==True)[0]
    diffs = idx[1:]-idx[:-1]
    idx2 = np.where(np.logical_and(diffs>1, diffs<min_seg_len))[0]
    #print ("short segs deletion: ", idx2.shape)

    # loop over short segs and remove them
    for id_ in idx2:
        data[idx[id_]:idx[id_+1]]=np.nan

    # VELOCITIES
    if True:

        if False:
            plt.scatter(np.arange(data.shape[0])/25, 
                    data.sum(1), color='black')
            
        #temp = temp1[1:]-temp1[:-1]

        locs_temp = np.sqrt(data[:,0]**2+data[:,1]**2)
        velocity = locs_temp[1:]-locs_temp[:-1]

        # absolute velocity delete
        idx = np.where(np.abs(velocity)>velocity_max)[0]
        data[idx]=np.nan

        # relative velocity delete
        idx = np.where(np.logical_and(np.abs(velocity[1:])>np.abs(velocity[:-1])*velocity_rel_max,
                                     np.abs(velocity[1:])>velocity_min))[0]

        #print ("relative vellcity deletions ", idx.shape)
        data[idx]=np.nan   
        
        idx = np.where(np.logical_and(np.abs(velocity[1:])*velocity_rel_max<np.abs(velocity[:-1]),
                                     np.abs(velocity[1:])>velocity_min))[0]

        #print ("relative vellcity deletions ", idx.shape)
        data[idx]=np.nan   

        

    # DEL SHORT SEGS 
    idx = np.where(np.isnan(data[:,0])==True)[0]
    diffs = idx[1:]-idx[:-1]
    idx2 = np.where(np.logical_and(diffs>1, diffs<min_seg_len))[0]
    #print ("short segs deletion: ", idx2.shape)

    # loop over short segs and remove them
    for id_ in idx2:
        data[idx[id_]:idx[id_+1]]=np.nan
        
    locs_array[animal_id] = data

    return locs_array

def connect_CSUA(locs_array,
                       animal_id,
                       max_jump, 
                       max_jump_intra_animal,     
                       min_nearby_animal_dist):
    
    animal_ids = np.arange(4)
    
    data = locs_array[animal_id].copy()
    #print ("Data: ", data.shape)
    
    
    idx = np.where(np.isnan(data[:,0])==False)[0]
   # print ("interpo: ", idx.shape)
   # print (idx[:10])
    diffs= idx[1:]-idx[:-1]
    idx2 = np.where(diffs>1)[0]
    #print (idx[idx2[:10]])
    
    idxnan = np.where(np.isnan(data[:,0])==True)[0]
    diffsnan = idxnan[1:]-idxnan[:-1]
    idx2nan = np.where(diffsnan>1)[0]
    
    for k in range(1,idx2.shape[0]-1,1):
        start = idx[idx2[k]]
        duration = diffs[idx2[k]]
        end = idxnan[idx2nan[k+1]]+1
        
        dist = np.sqrt((data[start,0]-data[end,0])**2+
                       (data[start,1]-data[end,1])**2)
        
        # CHECK MIN DIST; SET THIS TO CONSERVATIVE VALUE
        if dist <= max_jump:
                
            #print (data[start,0], data[end,0], duration)
            data[start:end,0]= np.linspace(data[start,0], data[end,0], duration)
            data[start:end,1]= np.linspace(data[start,1], data[end,1], duration)
            
        # CHECK MORE BROADLY ENSURE NO OTHER ANIMAL NEARBY
        elif dist <= max_jump_intra_animal:
            for a in animal_ids:
                if a == animal_id:
                    continue
                    #dists_start.append(1E10)
                    #dists_end.append(1E10)
                    
                near_anim = np.sqrt((locs_array[a][start,0]-locs_array[animal_id][start,0])**2+
                                      (locs_array[a][start,1]-locs_array[animal_id][start,1])**2)
                
                if np.min(near_anim)<min_nearby_animal_dist:
                    continue
                    
                near_anim = np.sqrt((locs_array[a][end,0]-locs_array[animal_id][end,0])**2+
                                      (locs_array[a][end,1]-locs_array[animal_id][end,1])**2)
                
                if np.min(near_anim)<min_nearby_animal_dist:
                    continue

            #print (data[start,0], data[end,0], duration)
            data[start:end,0]= np.linspace(data[start,0], data[end,0], duration)
            data[start:end,1]= np.linspace(data[start,1], data[end,1], duration)
            
    locs_array[animal_id] = data
    
    return locs_array



def swap_traces(locs_array, 
                animal_id,
                animal_ids):
    '''  Function that tries to correct obvious swaps
    '''
    
    # GRAB CSUAs FOR ANIMAL AND SEE IF BETTER MATCH WITH OTHER STARTS/ENDS
    # GRAB CSUAs from main animal
    idx = np.where(np.isnan(locs_array[animal_id])==False)[0]
    diffs = idx[1:]-idx[:-1]
    idx_end = np.where(diffs>1)[0]
    
    idxnan = np.where(np.isnan(locs_array[animal_id])==True)[0]
    diffs = idxnan[1:]-idxnan[:-1]
    idx_start = np.where(diffs>1)[0]
    
    CSUA = np.array([idxnan[idx_start],
                    idx[idx_end]]).T
    print ("animal id: ", animal_id, 
           "CSUA start: ", CSUA)
    
    # FIND WHICH CSUAS HAVE LARGE JUMPS FROM NEIGHBOURS
    
    
#     # FIND CSUAs for other animals
#     for a in animal_ids:
#         idx = np.where(np.isnan(locs_array[a])==False)[0]
#         diffs = idx[1:]-idx[:-1]
#         idx2 = np.where(diffs>1)[0]    
    
    
    return locs_array



def cleanup(fname_classification, 
            animal_arrays,
            animal_ids,
            locs_array,
            velocity_max = 200,
            velocity_min = 10.,
            velocity_rel_max = 7.,
            min_seg_len = 5,
            max_jump = 100, # SAFE DISTANCE TO MERGE SPLIT 
            max_jump_intra_animal = 500, # DISTANCE TO MERGE IF NO OTHER ANIMALS "NEARBY"
            min_nearby_animal_dist = 30, # DISTANCE DEFINING "NEARBY"
            separation_dist = 100, # Distance under which labels can no longer change location
           ):

    for animal_id in animal_ids:
        #print (locs_array[animal_id].shape)

        ###################################################
        ################## CLEANUP TRACES #################
        ###################################################
        if True:
            locs_array = cleanup_traces(locs_array,
                              animal_id,
                              velocity_min,
                              velocity_min,
                              velocity_rel_max,
                              min_seg_len)

        ###################################################
        ############# SWAP TRACES - NOT IMPLEMENTED #######
        ###################################################
    #     locs_array = swap_traces(locs_array,
    #                              animal_id,
    #                              animal_ids)

        ###################################################
        ############## CONNECTS CSUAS  ####################
        ###################################################
        if True:
            # REPEAT INTERPOLATION X TIMES
            for k in range(5):
                locs_array = connect_CSUA(locs_array,
                                                animal_id,
                                                max_jump,
                                                max_jump_intra_animal,     
                                                min_nearby_animal_dist)
        ###################################################
        ################# INTERPOLATE TRACES ##############
        ###################################################
        if True:
            # Fix x,y positions for missing data to last value in time
            # TODO: Try to recover missing data by looking at 
            #  raw heatmaps or peaks outputed by DLC
            locs_array = interpolate_traces(locs_array,
                                            animal_id)
        
        
        ###################################################
        ######## STOPS TRACES FROM JUMPING/JITTER #########
        ###################################################
        locs_array = stop_traces_jumps(locs_array, separation_dist)

            

    return locs_array 

def interpolate_traces(locs_array,
                       animal_id):
    
    ''' Replace all np.nans by the last good value
    '''
        
    data = locs_array[animal_id].copy()
    #print ("Data: ", data.shape)
    
    idx = np.where(np.isnan(data)==True)[0]
    for id_ in idx:
        data[id_]=data[id_-1]
    
    # go backwoards in time to ensure beginning of time also has non np.nan vals
    data = data[::-1]
    idx = np.where(np.isnan(data)==True)[0]
    for id_ in idx:
        data[id_]=data[id_-1]

    data = data[::-1]
    locs_array[animal_id] = data
    
    return locs_array

def stop_traces_jumps(locs_array, separation_dist):
    ''' Function that freezes traces once an animal gets near another
    '''
    
    # COMPUTE INTER-ANIMAL DISTANCES AT EVERY POINT:
    
    for a in range(4):
        
        locs_a = locs_array[a]
        
        for aa in range(4):
            if a==aa:
                continue
            locs_aa = locs_array[aa]
            
            dists = np.sqrt((locs_a[:,0]-locs_aa[:,0])**2+
                            (locs_a[:,1]-locs_aa[:,1])**2)
            
            idx = np.where(dists<separation_dist)[0]
            # search forward in time and fix to last value if too close 
            for id_ in idx:
                locs_a[id_]=locs_a[id_-1]

    
    return locs_array

In [10]:
#################################################################
###### GET HUNGARIAN ALGORITHM ASSEMBLED DATA WITHOUT VIDEO #####
#################################################################

# Classifier output for CSUA; using 100 x 100 pixel images;
fname_classification = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/classification_output.npz'
                        #/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/classification_output.npz

# Notebook 11 - CC based image extraction generates chain_id file 
fname_chain_ids = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences_0_89988_chain_id.npy'
chain_ids = np.load(fname_chain_ids)
print (chain_ids.shape)

# CSUA image locations; required for hungarian; may wish to simplifiy;
root_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/'

# Traces post-CC step
fname_traces_inferences = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences_0_89988.npz'
                          #'/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences_0_89988.npz'

comments=False
n_networks = 5
animal_ids, animal_ids_hungarian = get_animal_ids_function(fname_classification,
                                                  fname_traces_inferences,
                                                   chain_ids,
                                                   root_dir,
                                                   comments,
                                                   n_networks)
print ("DONE computing hungarian ID assignments")

(89988, 5)
DONE making animal_arrays:  (5, 89988, 2)
DONE computing hungarian ID assignments


In [13]:
##############################################################
####### COMPUTE SINGLE FEATURE TRACES POST HUNGARIAN #########
##############################################################
features = np.array(6, 'int32') # spine_1
ids = np.arange(5)
#ids=[0]

locs_array = get_single_feature_traces(fname_classification,
                                      fname_traces_inferences,
                                      animal_ids_hungarian,
                                      n_networks,
                                      features)

print ("DONE with single feature post huganrian", locs_array.shape)

DONE with single feature post huganrian (5, 89988, 2)


In [14]:
#################################################################
###### CLEANUP: Del short segs; connect safe segs ###############
#################################################################
#clrs = ['red','blue','cyan','green','yellow']
clrs = ['blue','red','yellow','green','yellow']

    
#print (locs_array.shape)
animal_arrays = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/animal_arrays.npy')
#print (animal_arrays.shape)
animal_arrays = locs_array.copy()

animal_ids = np.arange(5)
locs_array_clean = cleanup(fname_classification, 
                            animal_arrays,
                            animal_ids,
                            locs_array)
            
np.save(os.path.split(fname_classification)[0]+'/locs_array_clean.npy', 
                       locs_array_clean)

print ("DONE Post-processing traces - GO TO Notebook 15_behaviour_analysis")

/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/ipykernel_launcher.py:235: RuntimeWarning: invalid value encountered in greater
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/ipykernel_launcher.py:239: RuntimeWarning: invalid value encountered in greater
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/ipykernel_launcher.py:240: RuntimeWarning: invalid value encountered in greater
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/ipykernel_launcher.py:245: RuntimeWarning: invalid value encountered in less
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/ipykernel_launcher.py:246: RuntimeWarning: invalid value encountered in greater
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/ipykernel_launcher.py:474: RuntimeWarning: invalid value encountered in less


DONE Post-processing traces - GO TO Notebook 15_behaviour_analysis


In [14]:
###########################################################
###### CONVERT DATA TO JAABA trx.mat FORMAT ###############
###########################################################

def convert_npy_to_mat_for_jaaba(fname, 
                                 sexes,
                                 start,
                                 end,
                                dtype,
                                animal_ids,
                                 make_video
                                ):
    
    from scipy.io import savemat
    
    data = np.load(fname)
    
    if end is None:
        end = data.shape[1]
        
    trx_array=[]
    for k in animal_ids:
        x = np.array(data[k,start:end,0], dtype) # x-coordinate of the animal in pixels (1 x nframes).
        y = np.array(data[k,start:end,1], dtype) # y-coordinate of the animal in pixels (1 x nframes).
        
        print (' # of data points: ', x.shape)
        # rotations and body shrinking/stretching
        theta = x*0  # TO BE RECOMPUTED FROM FULL FEATURES:  Orientation of the animal (head) (1 x nframes); 
        a = x*0+100  # TO BE RECOMPUTED FROM FULL FEATURES:  1/4 of the major-axis length in pixels (1 x nframes).
        b = x*0+50  # TO BE RECOMPUTED FROM FULL FEATURES:  1/4 of the minor-axis length in pixels (1 x nframes).

        # meta data for the video and animal
        nframes = np.array(end-start,dtype)    # Number of frames in the trajectory of the current animal (scalar).
        firstframe = np.array(1,dtype)             # First frame of the animal's trajectory (scalar). USE MATLAB INDEXES
        endframe = np.array(nframes,dtype)   # Last frame of the animal's trajectory (scalar).
        off = 0                    # Offset for computing index into x, y, etc. Always equal to 1 - firstframe (scalar).
        id_ = k+1                  # Identity number of the trajectory (scalar). USE MATLAB VALUES

        # convert from pixels to distances
        pixels_per_mm = 2                       # number of pixels per mm # to figure out
        x_mm = x//pixels_per_mm                   # x-coordinate of the animal in mm (1 x nframes).
        y_mm = y//pixels_per_mm                   # y-coordinate of the animal in mm (1 x nframes).

        theta_mm = theta                     # Orientation of the animal in real coordinates. This is often the same as theta, if no transformation other than translation and scaling is performed between pixels and real coordinates (1 x nframes).

        a_mm = a//pixels_per_mm  # 1/4 of the major-axis length in mm (1 x nframes).
        b_mm = b//pixels_per_mm  # 1/4 of the major-axis length in mm (1 x nframes).

        sex = sexes[k]  # Sex of the animal. Can be just one value ('M' or 'F' or '?') or a cell array of 'M' and 'F' giving the sex for each frame. The size of the cell array should be nframes.

        # frame-rate based info
        dt = np.ones(int(nframes)-1, dtype)   # Difference in timestamps of the current frame and next frame, in seconds (1 x nframes-1).
        fps = 25                               # Average frames-per-second (scalar).

        days_per_timestamp = 1./(24*60*60*fps)
        timestamps = np.array(np.arange(nframes)*days_per_timestamp,
                             dtype) # Timestamp of each frame (optional), in days (1 x nframes). 

        
        trx ={
         'x': x,
         'y':y,
         'theta': theta,
         'a': a,
         'b': b,
         'nframes':nframes,
         'firstframe':firstframe,
         'endframe':endframe,
         'off':off,
         'id':id_,
         'x_mm':x_mm,
         'y_mm':y_mm,
         'theta_mm':theta_mm,
         'a_mm':a_mm,
         'b_mm':b_mm,
         'sex':sex,
         'dt':dt,
         'fps':fps,
         'timestamps': timestamps
        }
        
        trx_array.append(trx)

    trx_dict = {'a':trx_array[0],
                'b':trx_array[1],
                'c':trx_array[2],
                'd':trx_array[3]               
               }
        
    
    def split_movie(video_name, start, end):
        from tqdm import trange

        fname_out = video_name[:-4]+"_"+str(start)+"_"+str(end)+".mp4"
        if os.path.exists(fname_out):
            return

        #video_name = root_dir + '/' + os.path.split(root_dir)[1]+'.avi'
        original_vid = cv2.VideoCapture(video_name)

        # video sizes
        size_vid = np.array([1280,1024])
        original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

        # Initialize video out
        fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
        video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0],size_vid[1]), True)

        for n in trange(start,end, 1):
            ret, frame = original_vid.read()
            video_out.write(frame)
        video_out.release()
        original_vid.release()

    if make_video:
        video_name = os.path.split(fname)[0]+"/movie.avi"

        # 
        split_movie(video_name,start,end)
    
    
    savemat(fname[:-4]+'_trx.mat', 
            {
             'trx':trx_dict
            }
           )
    
    
    #
    print ("*** DONE CONVERSION ***")
    
fnames = glob.glob('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/post_processing/centres/'+
                   '*.npy')

for fname in fnames:
    sexes = ['M','M','M','M']
    sexes = np.ones(4)
    fps = 25
    start = int(13.5*60*fps)
    end = int(15.3*60*fps)

    start = 0
    end = None

    dtype = 'double'
    animal_ids = np.arange(4)
    make_video = False


    convert_npy_to_mat_for_jaaba(fname, 
                                 sexes,
                                 start,
                                 end,
                                 dtype,
                                 animal_ids,
                                 make_video)
    
    
    

 # of data points:  (89985,)
 # of data points:  (89985,)
 # of data points:  (89985,)
 # of data points:  (89985,)
*** DONE CONVERSION ***
 # of data points:  (89988,)
 # of data points:  (89988,)
 # of data points:  (89988,)
 # of data points:  (89988,)
*** DONE CONVERSION ***
 # of data points:  (89976,)
 # of data points:  (89976,)
 # of data points:  (89976,)
 # of data points:  (89976,)
*** DONE CONVERSION ***
 # of data points:  (89987,)
 # of data points:  (89987,)
 # of data points:  (89987,)
 # of data points:  (89987,)
*** DONE CONVERSION ***
 # of data points:  (89988,)
 # of data points:  (89988,)
 # of data points:  (89988,)
 # of data points:  (89988,)
*** DONE CONVERSION ***
 # of data points:  (89988,)
 # of data points:  (89988,)
 # of data points:  (89988,)
 # of data points:  (89988,)
*** DONE CONVERSION ***
 # of data points:  (89987,)
 # of data points:  (89987,)
 # of data points:  (89987,)
 # of data points:  (89987,)
*** DONE CONVERSION ***
 # of data points:  

In [13]:
from tqdm import trange
clrs = ['red','blue','green','cyan']

dirs = np.sort(glob.glob('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/trx_files/'+
                "*"))

imgs=[]
imgs1 = []
for dir_ in dirs:
    fname_mat = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/trx_files/2020-3-16_01_54_23_358257_compressed.avi.predictions.analysis_traces_reassembled_centres/scores_run.mat'
    data =loadmat(fname_mat)
    
    # get length
    temp = data['allScores'][0][0][0][0][0].squeeze()
    length = temp.shape[0]

    # get starts/ends
    t0s = data['allScores'][0][0][5].squeeze()
    #print (t0s.shape)
    t1s = data['allScores'][0][0][6].squeeze()
    
    img = np.zeros((4,length),'float32')
    img1 = np.zeros((4,length),'float32')
    for k in trange(t0s.shape[0]):
    #for k in [0]:
        starts = t0s[k].squeeze()
        ends = t1s[k].squeeze()
        #print (k,starts.shape)
        for p in range(starts.shape[0]):
            img[k,starts[p]:ends[p]]+=k+1
            img1[k,starts[p]:ends[p]]+=1
            
    idx = np.where(img==0)
    img[idx]=np.nan
    imgs.append(img)
    imgs1.append(img1)
    
imgs=np.hstack(imgs)
imgs1=np.hstack(imgs1)


100%|██████████| 4/4 [00:00<00:00, 15.94it/s]


In [18]:
matplotlib.use('Agg')

fig = plt.figure(figsize=(100,5))
ax=plt.subplot(2,1,1)


plt.xticks([])
print (imgs.shape)
names = ['female','male','pup1','pup2']
clrs = ['red','blue','magenta','green']
plt.yticks(np.arange(4),names)
from matplotlib import colors
cmap = colors.ListedColormap(clrs)

plt.imshow(imgs,aspect='auto',interpolation='none',
          cmap=cmap)

ax=plt.subplot(2,1,2)
sums = np.nansum(imgs1, axis= 0)
print (sums.shape)

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    #y_smooth = np.convolve(y, box, mode='same')
    y_smooth = np.convolve(y,np.ones(box_pts,dtype=int),'same')
    
    return y_smooth

window = 60
t = np.arange(imgs.shape[1])/25./window
img2 = np.zeros((4,imgs.shape[1]))
for k in range(4):
    temp = imgs1[k]
    temp = smooth(temp,25*window)    
    #img2[k]=temp
    #temp = smooth(temp,25)    
    
    plt.plot(t,temp,clrs[k])

#plt.imshow(img2,aspect='auto',interpolation='none')#    cmap=cmap)    
plt.xticks(np.arange(0,600,10))
plt.xlim(t[0],t[-1])
plt.xlabel("Time (minutes)")
plt.suptitle("Run behaviors for all animals ("+str(window)+ "sec smoothing window)", fontsize=20)
plt.savefig('/home/cat/fig.png')
#plt.show()



(4, 899880)
(899880,)


In [100]:
temp = data['allScores'][0][0][0][0][k].squeeze()
print(temp.shape)


(89988,)


In [63]:
t0s = data['allScores'][0][0][5].squeeze()
print (t0s.shape)
t1s = data['allScores'][0][0][6].squeeze()

(4,)


In [90]:
from tqdm import trange
clrs = ['red','blue','green','cyan']
#fig=plt.figure()
#ax=plt.subplot(111)
img = np.zeros((4,90000),'float32')
for k in trange(t0s.shape[0]):
#for k in [0]:
    starts = t0s[k].squeeze()
    ends = t1s[k].squeeze()
    print (k,starts.shape)
    for p in range(starts.shape[0]):
        img[k,starts[p]:ends[p]]+=k+1
idx = np.where(img==0)
img[idx]=np.nan
plt.imshow(img,aspect='auto',interpolation='none')
plt.show()
    
    
    

100%|██████████| 4/4 [00:00<00:00, 30.02it/s]


0 (6211,)
1 (6640,)
2 (6263,)
3 (2571,)


In [167]:
fname_mat = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/jaaba/trx_all_21300_21500.mat'
data = loadmat(fname_mat)

In [168]:
print (data['trx'].shape)
print (data['trx'][0][0][1].shape)

data['trx'][0][0][1][0][0] = 44
data['trx'][0][0][1][0] = np.array(data['trx'][0][0][1][0],dtype)

print (data['trx'][0][0][1][0][0])

(1, 4)
(1, 200)
44.0


In [140]:
print (data)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Fri Feb 12 16:27:47 2021', '__version__': '1.0', '__globals__': [], 'trx': array([[(array([[228.49064636, 226.56628418, 228.38729858, 230.02825928,
        228.97384644, 228.97384644, 234.30926514, 234.4354248 ,
        226.28547668, 226.35765076, 226.66925049, 226.56523132,
        226.26817322, 228.48426819, 227.97824097, 228.22874451,
        228.03605652, 234.43307495, 227.89006042, 228.12922668,
        228.01741028, 227.90434265, 227.74993896, 227.95742798,
        234.10432434, 227.46379089, 226.2374115 , 228.28546143,
        230.16645813, 228.33918762, 228.09303284, 228.21636963,
        228.06030273, 227.80683899, 225.70835876, 227.63092041,
        225.94082642, 226.38795471, 226.15658569, 228.41294861,
        228.17630005, 228.81712341, 236.18525696, 234.34925842,
        226.44261169, 228.04951477, 225.7795105 , 238.25611877,
        222.43948364, 224.57640076, 228.38641357, 228.157547  ,
        227.899

In [ ]:
  
    
from numpy.core.records import fromarrays
from scipy.io import savemat

names = [
 'x',
 'y',
 'theta',
 'a',
 'b',
 'nframes',
 'firstframe',
 'endframe',
 'off',
 'id',
 'x_mm',
 'y_mm',
 'theta_mm',
 'a_mm',
 'b_mm',
 'sex',
 'dt',
 'fps',
 'timestamps']

data = [
 x,
 y,
 theta,
 a,
 b,
 nframes,
 firstframe,
 endframe,
 off,
 id_,
 x_mm,
 y_mm,
 theta_mm,
 a_mm,
 b_mm,
 sex,
 dt,
 fps,
 timestamps
]


            
#myrec = fromarrays([[1, 10], [2, 20]], names=['field1', 'field2'])
#savemat('p.mat', {'myrec': myrec})

myrec = fromarrays(data, 
                   names=names)
#savemat('p.mat', {'myrec': myrec})


savemat("/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/jaaba/trx.mat", 
        {'myrec':myrec})
    

In [11]:
fnames = glob.glob('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/post_processing/*.npy')

for fname in fnames:
    data = np.load(fname)
    
    data2 = []
    for f in range(0,56,14):
        temp = data[:,f:f+14]
        temp = np.nanmedian(temp,axis=1)
        data2.append(temp)
        
    data2=np.array(data2)
    np.save(fname[:-4]+'_centres.npy',data2)
    print (data2.shape)

(4, 89988, 2)
(4, 89985, 2)
(4, 89987, 2)
(4, 89988, 2)
(4, 89987, 2)
(4, 89987, 2)
(4, 89988, 2)
(4, 89989, 2)
(4, 89976, 2)
(4, 89988, 2)


In [17]:
#  


# locs_array = get_single_feature_traces(fname_classification,
#                                       fname_traces_inferences,
#                                       animal_ids_hungarian)

# animal_id = 3
# idx = np.where(locs_array[animal_id,:,0]==0)[0]
# locs_array[animal_id,idx]=np.nan
# idx = np.where(np.isnan(locs_array[animal_id,:,0])==False)[0]
# print (idx.shape)

# plt.scatter(idx, locs_array[animal_id][idx].sum(1),c='black')

# # INTERPOLOATE STEP
# locs_array = interpolate_traces(locs_array,
#                                 animal_id)

# t= np.arange(locs_array[animal_id].shape[0])
# print (t.shape, locs_array[animal_id].shape)
# plt.scatter(t, locs_array[animal_id,:].sum(1)+50,c='blue')

# plt.show()

(27175,)
(89988,) (89988, 2)
